In [22]:
import requests
from bs4 import BeautifulSoup


url="https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"

response = requests.get(url)

if response.status_code==200:
    soup = BeautifulSoup(response.content,"html.parser")
    text= soup.get_text()
    with open("output.txt",'w',encoding='utf-8') as f:
        f.write(text)
        
    print("text has been written")
    
else:
    print("failed to write the content",requests.status_codes)
    

text has been written


In [23]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM


filepath = "./output.txt"
try:
    with open(filepath,'r',encoding='utf-8') as f:
        
        text = f.read().lower()
        print("the file was read")
        
except FileNotFoundError:
    print("the file was not found")
    
except Exception as e :
    print(f"an error has occured: {e}")

the file was read


In [24]:
te=text[300:8000000]
character=sorted(set(te))

In [26]:
# character

In [27]:
char_to_index= dict((c,i)for i,c in enumerate(character))

index_to_char= dict((i,c)for i,c in enumerate(character))